# Python连接MySQL，使用pymysql
- 安装pymysql
    -https://pymysql.readthedocs.io/en/latest/
- 创建测试数据库
    - 参考：http://blog.51cto.com/dnsliu/1349688
    - 参考：https://blog.csdn.net/u010472499/article/details/78208476
- 连接mysql，获取连接和游标
- 使用游标cursor的execute()方法向数据库发送sql语句
- 执行sql
    - 增：构造sql，使用游标的execute方法执行sql，提交事务
    - 删：构造sql，使用游标的execute方法执行sql，提交事务
    - 查：构造sql，使用游标的execute方法执行sql，fetchone,fetchmany,fetchall获取查询结果，提交事务
    - 改：构造sql，使用游标的execute方法执行sql，提交事务

## 0. 重要方法介绍

### 0.0 Connection对象-与MySQL的连接
表示一个和mysql server的socket，获取Connection实例的正确方法是调用connect()方法
```python
class pymysql.connections.Connection(host=None, user=None, password='', database=None, port=0, unix_socket=None, charset='', sql_mode=None, read_default_file=None, conv=None, use_unicode=None, client_flag=0, cursorclass=<class 'pymysql.cursors.Cursor'>, init_command=None, connect_timeout=10, ssl=None, read_default_group=None, compress=None, named_pipe=None, autocommit=False, db=None, passwd=None, local_infile=False, max_allowed_packet=16777216, defer_connect=False, auth_plugin_map=None, read_timeout=None, write_timeout=None, bind_address=None, binary_prefix=False, program_name=None, server_public_key=None)
```
- cursor()

cursor(cursor=None)
Create a new cursor to execute queries with.

Parameters:	cursor – The type of cursor to create; one of Cursor, SSCursor, DictCursor, or SSDictCursor. None means use Curs
- commit()

commit()
Commit changes to stable storage.

See Connection.commit() in the specification.


### 0.1 Cursor对象-与MySQL交互的对象：
这是与数据库交互的对象，不要使用Curosr的构造方法创建cursor，最好通过调用connections.Connection.cursor()来创建cursor

- execute方法
execute(query, args=None)
Execute a query

Parameters:	
query (str) – Query to execute.
args (tuple, list or dict) – parameters used with query. (optional)
Returns:	
Number of affected rows

Return type:	
int

If args is a list or tuple, %s can be used as a placeholder in the query. If args is a dict, %(name)s can be used as a placeholder in the query.

## 补充：事务
参考：http://www.runoob.com/mysql/mysql-transaction.html
参考：https://blog.csdn.net/xiaotom5/article/details/8133067
rollback，回滚，就是数据库做修改后（update，insert，delete）未commit之前，使用rollback，可以恢复到数据库修改之前。

## 1. 工作流程
1. 调用pymysql.connection()获取连接(参数方式，字典方式)
2. 调用connections.Connection.cursor()获取游标(with自动关闭，否则需要手动关闭)
3. 构造带参数的sql
4. 调用cursor.execute方法与数据库交互
5. 调用conn.commit方法提交
6. 关闭连接conn.close()

## 1.连接数据库
### 1.1 参数方式连接

In [1]:
import pymysql

conn = pymysql.connect(host='localhost',user='root',password='',db='',port=3306,charset='utf8')
#print(pymysql.connections.Connection.__doc__)
cur = conn.cursor()
#print(cur.__doc__)


### 1.2 字典方式连接

In [42]:
import pymysql.cursors
 
config = {
          'host':'127.0.0.1',
          'port':3306,
          'user':'root',
          'password':'',
          'db':'employees',
          'charset':'utf8',
          'cursorclass':pymysql.cursors.DictCursor,
          }
 
# Connect to the database
conn = pymysql.connect(**config)
print(conn)

## 2. 获取游标与增加记录



In [43]:
from datetime import date, datetime, timedelta
import pymysql.cursors
 
config = {
          'host':'127.0.0.1',
          'port':3306,
          'user':'root',
          'password':'',
          'db':'employees',
          'charset':'utf8',
          'cursorclass':pymysql.cursors.DictCursor,
          }
 
# Connect to the database
conn = pymysql.connect(**config)
# 获取明天的时间
tomorrow = datetime.now().date() + timedelta(days=1)
 
# 执行sql语句
try:
    with conn.cursor() as cursor:
        # 执行sql语句，插入记录
        sql = 'INSERT INTO employees (emp_no,first_name, last_name, hire_date, gender, birth_date) VALUES (%s,%s, %s, %s, %s, %s)'
        print(sql)
        t =  (1001,'Sheldon', 'Wang', tomorrow, 'M', date(1992, 6, 28))
        cursor.execute(sql,t);
    # 使用连接提交，没有设置默认自动提交，需要主动提交，以保存所执行的语句
    conn.commit()
 
finally:
    conn.close();

INSERT INTO employees (emp_no,first_name, last_name, hire_date, gender, birth_date) VALUES (%s,%s, %s, %s, %s, %s)


## 3.获取游标与删除记录 

In [48]:
import pymysql.cursors
import datetime
 
config = {
          'host':'127.0.0.1',
          'port':3306,
          'user':'root',
          'password':'',
          'db':'employees',
          'charset':'utf8',
          'cursorclass':pymysql.cursors.DictCursor,
          }
 
# Connect to the database
conn = pymysql.connect(**config)


# 获取雇佣日期
hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(2016, 12, 31)
 
# 执行sql语句
try:
    with conn.cursor() as cursor:
        # 执行sql语句，进行查询
        sql = 'delete from employees where emp_no = %s'
        t =  (100)
        cursor.execute(sql, t)
    # 没有设置默认自动提交，需要主动提交，以保存所执行的语句
    conn.commit()
 
finally:
    conn.close();

## 4. 获取游标与查询记录

In [50]:
import pymysql.cursors
import datetime
 
config = {
          'host':'127.0.0.1',
          'port':3306,
          'user':'root',
          'password':'',
          'db':'employees',
          'charset':'utf8',
          'cursorclass':pymysql.cursors.DictCursor,
          }
 
# Connect to the database
conn = pymysql.connect(**config)


# 获取雇佣日期
hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(2016, 12, 31)
 
# 执行sql语句
try:
    with conn.cursor() as cursor:
        # 执行sql语句，进行查询
        sql = 'SELECT first_name, last_name, hire_date FROM employees WHERE hire_date BETWEEN %s AND %s'
        t =  (hire_start, hire_end)
        cursor.execute(sql, t)
        # 获取查询结果
        result = cursor.fetchone()
        print(result)
    # 没有设置默认自动提交，需要主动提交，以保存所执行的语句
    #conn.commit()
 
finally:
    conn.close();

{'first_name': 'Lillian', 'last_name': 'Haddadi', 'hire_date': datetime.date(1999, 4, 30)}


## 5. 获取游标与修改记录

In [49]:
import pymysql.cursors
import datetime
 
config = {
          'host':'127.0.0.1',
          'port':3306,
          'user':'root',
          'password':'',
          'db':'employees',
          'charset':'utf8',
          'cursorclass':pymysql.cursors.DictCursor,
          }
 
# Connect to the database
conn = pymysql.connect(**config)


# 获取雇佣日期
hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(2016, 12, 31)
 
# 执行sql语句
try:
    with conn.cursor() as cursor:
        # 执行sql语句，进行查询
        sql = 'update employees set last_name = %s where emp_no = %s'
        t =  ('Lee','1001')
        cursor.execute(sql, t)

    # 没有设置默认自动提交，需要主动提交，以保存所执行的语句
    conn.commit()
 
finally:
    conn.close();